In [1]:
%matplotlib inline

In [2]:
import tensorflow as tf
from math import *
import numpy as np
from random import shuffle
from pylab import *
from multiprocessing import Pool

In [3]:
REG_DIM = 300 # dimension of logistic regression


In [11]:
# get labels
from random import shuffle

def read_user_labels():
    user_labels = {}

    for fn in ['contest_dataset_label/label000000_0', 'contest_dataset_label/label000001_0']:
        with open(fn) as file_user_label:
            for x in file_user_label:
                user_id, gender, user_age = x.split()
                user_labels[user_id] = [int(gender) - 1, int(user_age) - 1]

    return user_labels

user_labels = read_user_labels()
print len(user_labels)

11439408


In [4]:
#get testset
testset = []
with open('contest_testset/testset000000_0') as file_test:
    for x in file_test.readlines():
        testset += [x[:-1]]


In [13]:
#get url
MAX_URL = 2000000
def load_url(params):
    lo, hi = params
    gender, age = np.zeros((MAX_URL, 2), np.int32), np.zeros((MAX_URL, 7), np.int32)
    for i in range(lo, hi):
        fn = 'contest_dataset_url/part-r-%05d' % (i)
        print fn
        with open(fn) as fin:
            D = {}
            for x in fin:
                user_id, app_id, time = x.split()
                app_id = int(app_id) - 1
                if user_id not in user_labels:
                    continue
                if user_id not in D:
                    D[user_id] = {}
                if app_id not in D[user_id]:
                    D[user_id][app_id] = True
                    
                    g, a = user_labels[user_id]
                    gender[app_id][g] += 1
                    age[app_id][a] += 1
    return gender, age

In [8]:
pool = Pool(25)
# app_url = load_url([0, 8])

params = []
total = 179
start = 0
for i in range(25):
    x = (total - start) // (25 - i)
    params += [(i, start, start + x)]
    start += x
# print params

# all_results = pool.map(load_url, params)

In [16]:
freq_gender, freq_age = np.zeros((MAX_URL, 2), np.int32), np.zeros((MAX_URL, 7), np.int32)

for i, [gender, age] in enumerate(all_results):
    print i
    freq_gender += gender
    freq_age += age

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [7]:
T = np.sum(freq_gender, axis=1)
# np.save('tmp_url/freq', {'gender': freq_gender, 'age': freq_age})

In [6]:
x = np.load('tmp_url/freq.npy').item()
freq_gender, freq_age = x['gender'], x['age']

In [8]:
print T[T > 10000].shape
print sum(T != 0)
for i in range(1, 10):
    print i, sum(T == i)
print max(T)
# figure()
# hist(log(T[T > 10000]), bins=10)
# show()
# for i in xrange(MAX_URL):
#     if T[i] > 10000:
#         print 1. * freq_gender[i] / sum(freq_gender[i])

(2906,)
1845409
1 535978
2 295078
3 214769
4 125432
5 83116
6 60357
7 46886
8 37164
9 31446
4844361


In [1]:
#
all_gender, all_age = np.zeros(2), np.zeros(7)

for user_id in user_labels:
    gender, age = user_labels[user_id]
    all_gender[gender] += 1
    all_age[age] += 1

all_gender /= sum(all_gender)
all_age /= sum(all_age)

np.save("all_dist", {'gender': all_gender, 'age': all_age})

NameError: name 'np' is not defined

In [11]:
good_urls = np.where(T > 10000)[0]

In [12]:
'''
determine `good` app_id.

criterion: 
  1. number of users must be larger than MIN_USERS
  2. (DEPRECATED) corresponding distribution is `significantly` far from overall_distribution
     `significantly`: distance > AGE_THRES (for age) or distance > GENDER_THRES (for gender)
     distance measurement: KL divergence
  3. picking the apps with largest distance from overall distribution.
'''

MIN_USERS = 10000

gender_dists = []
age_dists = []
for url_id in good_urls:
    gender, age = freq_gender[url_id], freq_age[url_id]
    if sum(gender) < MIN_USERS:
        continue
    population = sum(gender)
    gender = 1. * gender / population
    age = 1. * age / population
    
    extra = log(population) * 0.001
    gender_dists += [(np.sum(gender * log(gender / all_gender)) + extra, url_id)]
    age_dists += [(np.sum(age * log(age / all_age)) + extra, url_id)]

gender_dists = list(reversed(sorted(gender_dists)))
age_dists = list(reversed(sorted(age_dists)))


In [4]:
good_urls = np.load("tmp_url/good_urls.npy").item()
good_gender_urls = good_urls['gender']
good_age_urls = good_urls['age']

In [5]:
'''
Save/load good apps from file
'''


# print len(gender_dists), len(age_dists)

# good_gender_urls = [(i, y) for i, (x, y) in enumerate(gender_dists[:REG_DIM])]
# good_age_urls = [(i, y) for i, (x, y) in enumerate(age_dists[:REG_DIM])]

# np.save('tmp_url/good_urls', {'gender': good_gender_urls, 'age': good_age_urls})

good_age = {}
good_gender = {}
for i, url_id in good_gender_urls:
    good_gender[url_id] = i # , freq_gender[url_id][0]

for i, app_id in good_age_urls:
    good_age[url_id] = i #, freq_age[url_id][1]

# for i in range(REG_DIM):
#     print good_gender_urls[i], np.sum(freq_gender[good_gender_urls[i][1]]), good_age_urls[i], np.sum(freq_age[good_age_urls[i][1]])

# figure()
# hist([x for x, y in gender_dists if x > 0.2], bins=40)
# show()
# print gender_dists[100]
# print age_dists[100]
# print log(7) / log(2) * 0.14591690295265844

In [6]:
# for each user, which urls does he visit? 

def load_user_url(params):
    global REG_DIM
    pid, lo, hi = params
    D = []
    total, useful = 0, 0
    for i in range(lo, hi):
        fn = 'contest_dataset_url/part-r-%05d' % (i)
        print fn
        with open(fn) as fin:
            for x in fin:
                user_id, app_id, time = x.split()
                app_id = int(app_id) - 1
                total += 1
#                 if user_id not in D:
#                     D[user_id] = np.zeros(REG_DIM, np.float32), np.zeros(REG_DIM, np.float32)
#                     D[user_id] = {}
                if app_id in good_gender or app_id in good_age:
                    useful += 1
                    D += [(user_id, app_id)]
#                     if app_id not in D[user_id]:
#                         D[user_id][app_id] = True
#                 if app_id in good_gender:
#                     D[user_id][good_gender[app_id][0]] = 1.
#                 if app_id in good_age:
#                     D[user_id][good_age[app_id][0]] = 1.
    np.save("tmp_url/feature-%d.npy" % (pid), D)
    print "ratio = %d/%d ~ %.6f" % (total, useful, 1. * useful / total)
#     return D

In [9]:
# all_results = pool.map(load_user_url, params)
load_user_url(params[0])

contest_dataset_url/part-r-00000
contest_dataset_url/part-r-00001
contest_dataset_url/part-r-00002


KeyboardInterrupt: 

In [25]:
good_gender

{194: 88,
 4666: 157,
 24515: 87,
 35095: 62,
 36303: 201,
 54790: 259,
 68674: 106,
 73129: 17,
 78847: 173,
 112176: 251,
 136981: 121,
 140918: 299,
 141036: 280,
 146752: 18,
 148279: 196,
 160920: 197,
 163498: 14,
 164321: 16,
 181599: 59,
 192935: 205,
 199048: 10,
 208817: 142,
 220248: 41,
 220399: 274,
 223993: 133,
 230199: 277,
 231239: 116,
 240356: 164,
 246570: 27,
 248385: 281,
 269698: 189,
 274165: 11,
 274350: 241,
 275577: 291,
 277696: 243,
 293252: 228,
 294556: 198,
 299628: 101,
 305135: 38,
 311784: 264,
 314265: 194,
 316552: 235,
 318870: 273,
 319424: 1,
 328035: 179,
 328813: 111,
 335718: 295,
 342715: 115,
 348873: 156,
 353131: 268,
 356611: 225,
 365260: 143,
 365656: 85,
 380190: 74,
 391081: 286,
 392187: 63,
 393941: 165,
 400644: 242,
 412513: 55,
 439629: 113,
 454216: 20,
 457750: 154,
 471533: 39,
 487710: 292,
 488310: 103,
 489948: 64,
 492547: 51,
 496554: 182,
 507555: 160,
 511234: 98,
 512195: 186,
 513571: 86,
 514166: 187,
 524308: 128,
 

In [12]:
len(user_labels)

11439408

In [13]:
np.save("labeled_user_id", user_labels)